In [1]:
import matplotlib.pylab as plt
import numpy as np
import uproot

plt.style.use( "/afs/ihep.ac.cn/users/l/luoxj/Style/Paper.mplstyle")
%config InlineBackend.figure_format = 'svg'
import sys
sys.path.append("/afs/ihep.ac.cn/users/l/luoxj/root_tool/python_script/")

In [ ]:
from importlib import reload
import LoadMultiFiles
reload(LoadMultiFiles)
from LoadMultiFiles import LoadOneFileUprootCertainEntries,LoadOneFileUproot
use_TimeTruth = False

# dir_calib = LoadOneFileUproot("/afs/ihep.ac.cn/users/l/luoxj/PSD_Supernova/myJUNOCommon/share/PSD/root_SaveTimeCharge/user_PSD_0__SN.root",
#                               name_branch="evt", return_list=False)
# dir_map = LoadOneFileUproot(f"/afs/ihep.ac.cn/users/l/luoxj/PSD_Supernova/myJUNOCommon/share/tag_event/sn_tag_0_save_TimeTruth.root",
#                             name_branch='evtTruth',list_branch_filter=["HitTypeTruth", "TimeoffsetTruth"],return_list=False)

dir_calib = LoadOneFileUprootCertainEntries("/afs/ihep.ac.cn/users/l/luoxj/PSD_Supernova/myJUNOCommon/share/PSD/root_SaveTimeCharge/user_PSD_0__SN.root",
                              name_branch="evt", return_list=False, n_entries_load=400)
dir_map = LoadOneFileUprootCertainEntries(f"/afs/ihep.ac.cn/users/l/luoxj/PSD_Supernova/myJUNOCommon/share/tag_event/sn_tag_0_save_TimeTruth.root",
                            name_branch='evtTruth',     return_list=False, n_entries_load=400)
#
# dir_calib = LoadOneFileUprootCertainEntries("/afs/ihep.ac.cn/users/l/luoxj/PSD_Supernova/myJUNOCommon/share/Sim_J21v1r0-Pre2/PSD/user_PSD_0__SN.root",
#                               name_branch="evt", return_list=False, n_entries_load=400)
# dir_map = LoadOneFileUprootCertainEntries(f"/afs/ihep.ac.cn/users/l/luoxj/PSD_Supernova/myJUNOCommon/share/Sim_J21v1r0-Pre2/tag_event/sn_tag_0.root",
#                             name_branch='evtTruth',     return_list=False, n_entries_load=400)


/afs/ihep.ac.cn/users/l/luoxj/root_tool/python_script/LoadMultiFiles.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dir_event[key] = np.array( list(tree[key]) )


In [ ]:
print(dir_map.keys(), dir_calib.keys())
v_tags = ["pES", "eES", "AfterPulse"]
print(len(dir_map["evtID"]), len(dir_calib["Time"]))
bins = np.arange(0,5000, 6)

In [ ]:
# Print into PDF
from collections import Counter
from matplotlib.backends.backend_pdf import PdfPages
if use_TimeTruth:
    path_save_pdf = "/afs/ihep.ac.cn/users/l/luoxj/PSD_Supernova/code/pdf/"
    for tag in v_tags:
        with PdfPages(f"{path_save_pdf}TimeTruth_{tag}.pdf") as pdf:
            index_tag =  (dir_map["evtType"]==tag)
            for i, (v_time, v_hitType) in enumerate (zip( dir_map["TimeTruth"][index_tag], dir_map["HitTypeTruth"][index_tag] ) ):
                plt.figure()
                for hitType in set(v_hitType):
                    plt.hist( v_time[v_hitType==hitType], bins=bins , histtype="step", label=hitType)
                # print( Counter(v_hitType) )
                plt.legend()
                plt.title(tag)
                pdf.savefig()
                plt.close()

                if i>30:
                    break


In [ ]:
import pandas as pd
print(dir_calib.keys())
print( len(dir_map["evtType"]), len(dir_calib["Time"]) )
dir_concat =  dict(dir_map, **dir_calib)
print(dir_concat.keys())

# for i in range(len( dir_concat["TimeTruth"] )):
#     dir_concat["TimeTruth"][i] = dir_concat["TimeTruth"][i]-min(dir_concat["TimeTruth"][i])


v_TotalCharge = []
for v_charge in dir_concat["Charge"]:
    v_TotalCharge.append( sum(v_charge) )
dir_concat["TotalCharge"] = np.array( v_TotalCharge )

df_concat = pd.DataFrame.from_dict(dir_concat)
df_concat = df_concat[(df_concat["TotalCharge"]>0) & (df_concat["TotalCharge"]<6000) & ( (df_concat["evtType"]=="AfterPulse") |
                                                           (df_concat["evtType"]=="pES") |
                                                           (df_concat["evtType"]=="eES") ) ]

df_concat

# Study Time Profile of MC Truth

## (1) Plot Time Profile

In [ ]:
from HistTools import GetBinCenter
bins = np.arange(0,5000, 6)
# bins=1000
bins_center = GetBinCenter(bins)

# v_tags = [ "AfterPulse"]
if use_TimeTruth:
    for tag in v_tags:
        plt.figure()
        for time in df_concat[df_concat["evtType"]==tag]["TimeTruth"][:5]:
            # plt.figure()
            plt.hist(time, bins=bins, histtype="step")
            plt.xlabel("Time[ns]")
        plt.title(tag)

# nPMTs Trigger Study
## (1) Calib Data
### Check Trigger Condition

In [ ]:
from TriggerTools import GetHitsCount
from PlotTools import LegendNotRepeated
# win_width = 80
win_width = 300
step_slide = 16

v_time_lowerLimit = np.arange(4, 600, step_slide)
v_time_upperLimit = v_time_lowerLimit+win_width
time_center = ( v_time_lowerLimit + v_time_upperLimit ) /2

v_strategy = ["nPMT_w/o_repeat", "nPMT_w/_repeat", "Charge"]
v_colors = ["r", "b", "g"]
v_ls = ["-", "--","-."]
print(v_tags)
for strategy in v_strategy:
    plt.figure(strategy)
for i, tag in enumerate (v_tags ):
    for index, row in list( df_concat[ df_concat["evtType"]==tag ].iterrows() )[:100]:
        dir_nHits = {key:[] for key in v_strategy}
        for time_lowerLimit, time_upperLimit in zip( v_time_lowerLimit, v_time_upperLimit ):
            for strategy in v_strategy:
                dir_nHits[strategy].append( GetHitsCount(row["PMTID"], row["Time"], row["Charge"],
                                     time_lowerLimit, time_upperLimit , strategy=strategy) )

        for j, strategy in enumerate( v_strategy ):
            plt.figure(strategy)
            # plt.step(time_center,dir_nHits[strategy], linewidth=2, label=strategy, color=v_colors[j], ls=v_ls[i])
            plt.plot(v_time_lowerLimit,dir_nHits[strategy], linewidth=1, label=tag, color=v_colors[i])

# plt.xlim(0,400)
for strategy in v_strategy:
    plt.figure(strategy)
    plt.ylim(0,1000)
    plt.xlabel("Time [ ns ]")
    plt.title(f"{win_width} ns Window with Step of {step_slide} ns")
    LegendNotRepeated()

    if strategy == "nPMT_w/o_repeat":
        plt.ylabel("N of Fired PMTs")
    elif strategy == "nPMT_w/_repeat":
        plt.ylabel("N of Hits")
    elif strategy == "Charge":
        plt.ylabel("Charge")



### Check Trigger Window Calculation

In [ ]:
v_strategy = ["nPMT_w/o_repeat"]
from matplotlib.backends.backend_pdf import PdfPages
path_save_pdf = "/afs/ihep.ac.cn/users/l/luoxj/PSD_Supernova/code/pdf/"

for i, tag in enumerate (v_tags ):
    with PdfPages(f"{path_save_pdf}nPMTFiredAndTimeProfile_{tag}.pdf") as pdf:
        for index, row in list( df_concat[ df_concat["evtType"]==tag ].iterrows() ):
            fig = plt.figure()
            dir_nHits = {key:[] for key in v_strategy}
            for time_lowerLimit, time_upperLimit in zip( v_time_lowerLimit, v_time_upperLimit ):
                for strategy in v_strategy:
                    dir_nHits[strategy].append( GetHitsCount(row["PMTID"], row["Time"], row["Charge"],
                                         time_lowerLimit, time_upperLimit , strategy=strategy) )

            for j, strategy in enumerate( v_strategy ):
                # plt.figure(strategy)
                fig.add_subplot(211)
                plt.bar(v_time_lowerLimit,dir_nHits[strategy], linewidth=1,width=win_width,align="edge", color=v_colors[i],
                        fill=False,ls="--")
                plt.step(v_time_lowerLimit,dir_nHits[strategy], linewidth=2, label=tag, color=v_colors[i],where="post")
                # plt.plot(v_time_lowerLimit,dir_nHits[strategy], linewidth=1, label=tag, color=v_colors[i])
                plt.ylabel("N of Fired PMTs")
                plt.title(f"{win_width} ns Window with Step of {step_slide} ns")
                LegendNotRepeated()

                fig.add_subplot(212)
                plt.hist(row["Time"], histtype="step", bins=np.arange(0, 600, 6),linewidth=1, label=tag, color=v_colors[i])
                plt.ylabel("w/o charge")
                plt.xlabel("Time [ ns ]")
                LegendNotRepeated()
            # pdf.savefig()
            # plt.close()
            break


## (2) Time Truth
### Check Whether Truth Align with Calib Data

In [ ]:
use_TimeTruth = True
bins_time = np.arange(-100, 1500, 6)
if use_TimeTruth:
    for tag in v_tags:
        for index, row in list( df_concat[df_concat["evtType"]==tag].iterrows() )[:5]:
            plt.figure()
            v_time_with_tag = row["PulseTimeTruth"]-row["TriggerTime"]+100
            # v_time_with_tag = row["TimeTruth"]
            v_time_calib_with_tag = row["Time"]
            v_charge_calib_with_tag = row["Charge"]
            print(v_time_with_tag)
            # delta_time = GetAlignValue(v_time_with_tag, v_time_calib_with_tag, v_charge_calib_with_tag, np.arange(0,5000, 6), align_method="threshold")
            plt.hist(v_time_with_tag
                     , bins=bins_time, histtype="step", label="Truth")

            plt.hist(v_time_calib_with_tag, bins=bins_time, histtype="step", label="Calib",weights=row["Charge"])

            plt.xlabel("Waveform Time [ ns ]")
            plt.legend()
            plt.title(tag)

            break


### Check Trigger Condition With Truth

In [ ]:
v_strategy = ["nPMT_w/o_repeat", "nPMT_w/_repeat"]

if use_TimeTruth:
    for strategy in v_strategy:
        plt.figure(strategy)
    for i, tag in enumerate ( v_tags ):
        for index, row in list( df_concat[ df_concat["evtType"]==tag ].iterrows() )[:30]:
            dir_nHits = {key:[] for key in v_strategy}
            for time_lowerLimit, time_upperLimit in zip( v_time_lowerLimit, v_time_upperLimit ):
                for strategy in v_strategy:
                    # index_hitType = (row["HitTypeTruth"]=="AfterPulse")
                    # dir_nHits[strategy].append( GetHitsCount(row["PMTIDTruth"][index_hitType], row["TimeTruth"][index_hitType], None,
                    v_time_with_tag = row["PulseTimeTruth"]-row["TriggerTime"]+100
                    dir_nHits[strategy].append( GetHitsCount(row["PMTIDTruth"], v_time_with_tag, None,
                                         time_lowerLimit, time_upperLimit , strategy=strategy) )

            for j, strategy in enumerate( v_strategy ):
                plt.figure(strategy)
                # plt.step(time_center,dir_nHits[strategy], linewidth=2, label=strategy, color=v_colors[j], ls=v_ls[i])
                plt.plot(v_time_lowerLimit,dir_nHits[strategy], linewidth=1, label=tag, color=v_colors[i])

    for strategy in v_strategy:
        plt.figure(strategy)
        plt.ylim(0,1000)
        plt.xlabel("PulseTime from Truth [ ns ]")
        plt.title(f"{win_width} ns Window with Step of {step_slide} ns")
        LegendNotRepeated()

        if strategy == "nPMT_w/o_repeat":
            plt.ylabel("N of Fired PMTs")
        elif strategy == "nPMT_w/_repeat":
            plt.ylabel("N of Hits")
        elif strategy == "Charge":
            plt.ylabel("Charge")

### Plot Contribution of Different Pulse Type

In [ ]:
import PlotTools
reload(PlotTools)
from PlotTools import PlotContributionOfEachArray
import tqdm

strategy = "nPMT_w/o_repeat"
if use_TimeTruth:
    for i, tag in enumerate ( v_tags ):
    # for i, tag in enumerate ( ["AfterPulse"] ):
        pdf = PdfPages(f"/afs/ihep.ac.cn/users/l/luoxj/PSD_Supernova/code/pdf/ContributionOfPulse_{tag}.pdf")
        for index, row in tqdm.tqdm( list( df_concat[ df_concat["evtType"]==tag ].iterrows() )[:20] ):
            plt.figure()
            v_hitType = set(row["HitTypeTruth"])
            v2d_NPMTs = []
            for time_lowerLimit, time_upperLimit in zip( v_time_lowerLimit, v_time_upperLimit ):
                v_time_with_tag = row["PulseTimeTruth"]-row["TriggerTime"]+100
                v_NPMTs_in_one_time_bin = []
                for PulseType in set(row["HitTypeTruth"]):
                    index_PulseType = (row["HitTypeTruth"]==PulseType)

                    v_NPMTs_in_one_time_bin.append( GetHitsCount(row["PMTIDTruth"][index_PulseType], v_time_with_tag[index_PulseType], None,
                                         time_lowerLimit, time_upperLimit , strategy=strategy) )
                v2d_NPMTs.append( v_NPMTs_in_one_time_bin )
            PlotContributionOfEachArray(v2d_NPMTs, label_index=list(v_time_lowerLimit), label_columns=list(v_hitType),show_table=False,reverse=True)
            plt.xlabel("Time [ ns ]")
            plt.ylabel("$Fired\ N_{PMTs}$")
            plt.title(f"{win_width} ns Window with Step of {step_slide} ns")
            
            pdf.savefig()
            plt.close()
        pdf.close()




In [ ]:
if use_TimeTruth:
    for tag in v_tags:
        for index, row in list( df_concat[df_concat["evtType"]==tag].iterrows() )[:10]:
            plt.figure()
            for pulseType in set(row["HitTypeTruth"]):
                v_time_with_tag = row["TimeTruth"][row["HitTypeTruth"]==pulseType]
                plt.hist(v_time_with_tag, bins=np.arange(0,2000,6), histtype="step", label=pulseType)
            plt.legend()
            plt.xlabel("Time [ ns ]")
            plt.title(tag)
